In [1]:
import pandas as pd
import os, openai
import time
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings


import string
LETTERS = string.ascii_uppercase

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

if os.getenv("OPENAI_API_KEY") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

underlying_embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))


store = LocalFileStore("./cache/")

embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)


OPENAI_API_KEY is ready


/tmp/ipykernel_31832/1528002886.py:11: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  underlying_embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))


In [3]:
if os.getenv("OAI") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))

df = pd.read_parquet('data/articles.parquet.gzip')
df.columns = ["src","content","LEN"]
df = df[(df.LEN > 1500) & (df.LEN < 30000)].reset_index(drop=True)
print(len(df),"articles of good lengths in the articles.partquet.gzip")
titles = pd.read_parquet("data/titles.parquet.gzip")
df = df.merge(titles, on="src",how="left")
mt = pd.read_parquet("data/metatags.parquet.gzip")
df = df.merge(mt,on="src",how="left")
df.to_parquet("data/consolidated.parquet.gzip",compression="gzip")
#df["text"] = df.content
df["source"] = df.url
df["author"] = df.origin.apply(lambda x: "kelu" if str(x).startswith("20") else "other")
for x in df.columns:
    df[x] = df[x].astype(str)
df_loader = DataFrameLoader(df, page_content_column="content")

df_document = df_loader.load()


OPENAI_API_KEY is ready
2039 articles of good lengths in the articles.partquet.gzip


In [8]:

text_splitter = CharacterTextSplitter(separator='\n\n',chunk_size=2000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(df_document)
print(len(chunked_documents),"docs to add")

base_path = "./DB/"
if os.getenv("OAI") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))


if not os.path.isfile(base_path+"chroma.sqlite3"):
    print("Start a new DB")
    vectordb = Chroma.from_documents(
        documents=[chunked_documents[0]],
        embedding=embeddings,
        persist_directory=base_path
    )
    vectordb.persist()
else:
    print("Continue on the DB")
    vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
    print(len(vectordb.get()["ids"]),"elements already stored.")
    LSDOCS = vectordb.get()["documents"]

print("Already",len(vectordb.get()["documents"]),"documents.")
print("Adding",len(chunked_documents),"documents.")

if vectordb:
    LSDOCS = vectordb.get()["documents"]
else:
    LSDOCS = []
    
for doc in chunked_documents:
    # Check if the text already exists somewhere
    if not doc.page_content in LSDOCS:
        vectordb.add_documents(
            documents=[doc], 
            embedding=embeddings, 
            persist_directory=base_path
        )
        # Ugly hack to avoid reaching token per min limit 
        # So it sleeps 1s between page
        time.sleep(0.001)
        vectordb.persist()
    else:
        0
        #print("Item already in the DB",doc.page_content[:100].replace("\n"," "))
LSDOCS = vectordb.get()["documents"]
vectordb.persist()

2039 docs to add
OPENAI_API_KEY is ready
Continue on the DB
2031 elements already stored.
Already 2031 documents.
Adding 2039 documents.
